In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [ ]:
train_df1 = pd.read_csv('/content/drive/MyDrive/symptoms_train.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df1.head()


,id,tweet,labels
0,1322367936405921794,uh oh the coughs have started covid is taking ...,non-reporting
1,1262153405553946624,"Loss of smell, confusion, strokes: Does Covid-...",non-reporting
2,1324014644931878912,"""AI is being used to distinguish between asymp...",non-reporting
3,1313660639097823232,i was v ill for the last couple of days (high ...,non-reporting
4,1239223445306695680,Played warzone yesterday with a dude who caugh...,third-party


In [ ]:
train_df1['non-reporting'] = 0

In [ ]:
train_df1['third-party'] = 0
train_df1['primary'] = 0
train_df1['secondary'] = 0

In [ ]:
print(train_df1.loc[[2]]['labels'] == 'non-reporting')

2    True
Name: labels, dtype: bool


In [ ]:
train_df1.head()

,id,tweet,labels,non-reporting,third-party,primary,secondary
0,1322367936405921794,uh oh the coughs have started covid is taking ...,non-reporting,0,0,0,0
1,1262153405553946624,"Loss of smell, confusion, strokes: Does Covid-...",non-reporting,0,0,0,0
2,1324014644931878912,"""AI is being used to distinguish between asymp...",non-reporting,0,0,0,0
3,1313660639097823232,i was v ill for the last couple of days (high ...,non-reporting,0,0,0,0
4,1239223445306695680,Played warzone yesterday with a dude who caugh...,third-party,0,0,0,0


In [ ]:
train_df1['non-reporting'] = train_df1.apply(lambda row: 1 if row.labels == 'non-reporting' else 0, axis=1)


In [ ]:
train_df1['secondary'] = train_df1.apply(lambda row: 1 if row.labels == 'secondary' else 0, axis=1)
train_df1['primary'] = train_df1.apply(lambda row: 1 if row.labels == 'primary' else 0, axis=1)
train_df1['third-party'] = train_df1.apply(lambda row: 1 if row.labels == 'third-party' else 0, axis=1)


In [ ]:
train_df1.head()

,id,tweet,labels,non-reporting,third-party,primary,secondary
0,1322367936405921794,uh oh the coughs have started covid is taking ...,non-reporting,1,0,0,0
1,1262153405553946624,"Loss of smell, confusion, strokes: Does Covid-...",non-reporting,1,0,0,0
2,1324014644931878912,"""AI is being used to distinguish between asymp...",non-reporting,1,0,0,0
3,1313660639097823232,i was v ill for the last couple of days (high ...,non-reporting,1,0,0,0
4,1239223445306695680,Played warzone yesterday with a dude who caugh...,third-party,0,1,0,0


In [ ]:
train_df1.drop('labels', axis=1, inplace=True)

In [ ]:
train_df1.head()

,id,tweet,non-reporting,third-party,primary,secondary
0,1322367936405921794,uh oh the coughs have started covid is taking ...,1,0,0,0
1,1262153405553946624,"Loss of smell, confusion, strokes: Does Covid-...",1,0,0,0
2,1324014644931878912,"""AI is being used to distinguish between asymp...",1,0,0,0
3,1313660639097823232,i was v ill for the last couple of days (high ...,1,0,0,0
4,1239223445306695680,Played warzone yesterday with a dude who caugh...,0,1,0,0


In [ ]:
target_list = ['non-reporting',	'third-party',	'primary',	'secondary']

In [ ]:
# for i in range(4392):
#   if(train_df.loc[[i]]['label'].values == 'ProVax'):
#     train_df.loc[[i]]['ProVax'] = 1
#   elif(train_df.loc[[i]]['label'].values == 'AntiVax'):
#     train_df.loc[[i]]['AntiVax'] = 1
#   else:
#     train_df.loc[[i]]['Neutral'] = 1


In [ ]:
# train_df['label']=train_df['label'].apply(lambda x: 1 if x=='ProVax' else 2 if x=='AntiVax' else 0 )


In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 13
LEARNING_RATE = 1e-05

In [ ]:
from transformers import BertTokenizer, BertModel


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['tweet']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        ) 

        return {
            
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }      





In [ ]:
train_size = 0.95
train_df = train_df1.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = train_df1.drop(train_df.index).reset_index(drop=True)

In [ ]:
train_df

,id,tweet,non-reporting,third-party,primary,secondary
0,1271932435790532611,Went ahead and took a COVID test as a precauti...,1,0,0,0
1,1268564176424759301,@AmericanAir my flight is being sent to a US A...,0,1,0,0
2,1280616072505245696,Of the 7 people I personally know with covid-1...,0,1,0,0
3,1281225112709148672,@Yorks94 @SooozeKerr @LPryceless @SkyNewsBreak...,0,1,0,0
4,1239282192729231362,Bought an 18-pack of corona. So far only hea...,0,0,1,0
...,...,...,...,...,...,...
1490,1246534133804916738,@mividovich @SVRaoMD @ARS_MD2004 @DrMauricioCo...,1,0,0,0
1491,1236475908724101120,"@realDonaldTrump, @BBCBreaking,@narendramodi S...",1,0,0,0
1492,1291849273685028864,@AlexBerenson My niece in law works at ER hosp...,0,0,0,1
1493,1237285908958887937,@ReneeStephen @HawksNest @EmmmieG @Eurocitizen...,1,0,0,0


In [ ]:
val_df

,id,tweet,non-reporting,third-party,primary,secondary
0,1311908999856492544,@NoLore my 8 year olds only symptoms of covid ...,0,0,0,1
1,1242016859840487424,"May be wrong, but I think taste and smell loss...",1,0,0,0
2,1253804618984947712,cant tell if i’ve got a cold and sore throat o...,0,0,1,0
3,1316737281571672064,@British_Megan Early December I had a dry coug...,0,0,1,0
4,1274842806155083776,"Thanks everyone, sorry for the pic I shared. O...",0,0,1,0
...,...,...,...,...,...,...
74,1247792485239156738,@CCriadoPerez Totally. Murphy’s nose was runni...,0,1,0,0
75,1295395700650528775,@kamera26 @Nicholaspkirby I just tested negati...,0,0,1,0
76,1241381235382984707,Lmao I have a cold or smth And every time I c...,0,0,1,0
77,1236525788360474629,"fuck the corona virus, all my homies got biebe...",0,0,0,1


In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
print(device)

cuda


In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()
def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)    

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 4)               #3 classes
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
val_targets=[]
val_outputs=[]

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))

    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      print(len(training_loader))
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
ckpt_path = "/content/drive/MyDrive/curr_ckpt_symp"
best_model_path = "/content/drive/MyDrive/best_model_symp.pt"

In [ ]:
# print(len(training_loader))

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
47
Epoch: 1 	Avgerage Training Loss: 0.011696 	Average Validation Loss: 0.156087
Validation loss decreased (inf --> 0.156087).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
47
Epoch: 2 	Avgerage Training Loss: 0.008676 	Average Validation Loss: 0.125659
Validation loss decreased (0.156087 --> 0.125659).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
###########

In [ ]:
example1 = 'Dont be complacent - young Pasifika urged to get vaccinated this weekend https://t.co/ahO54WqVkX https://t.co/Y7ECLdw4ew'

In [ ]:
example = '@TheMindsculpter I had a slight cold last week and my son has a chesty cough at the moment, he will not be getting tested for covid ðŸ¤«ðŸ˜‚ðŸ˜‚'

In [ ]:
encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

In [ ]:
model.eval()
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(final_output)
    print(train_df.columns[2:].to_list()[int(np.argmax(final_output, axis=1))])

[[0.06630554050207138, 0.03520825877785683, 0.07187189161777496, 0.9599885940551758]]
secondary


In [ ]:
df_again = pd.read_csv('/content/drive/MyDrive/symptoms_train.csv')
# testset = pd.read_csv('/content/drive/MyDrive/vax_train.csv')

In [ ]:
testset = df_again.sample(frac= 0.3, random_state=200).reset_index(drop=True)

In [ ]:
testset['non-reporting'] = testset.apply(lambda row: 1 if row.labels == 'non-reporting' else 0, axis=1)
testset['third-party'] = testset.apply(lambda row: 1 if row.labels == 'third-party' else 0, axis=1)
testset['primary'] = testset.apply(lambda row: 1 if row.labels == 'primary' else 0, axis=1)
testset['secondary'] = testset.apply(lambda row: 1 if row.labels == 'secondary' else 0, axis=1)



In [ ]:
len(testset)   #.size

472

In [ ]:
testset.head()

,id,tweet,labels,non-reporting,third-party,primary,secondary
0,1271932435790532611,Went ahead and took a COVID test as a precauti...,non-reporting,1,0,0,0
1,1268564176424759301,@AmericanAir my flight is being sent to a US A...,third-party,0,1,0,0
2,1280616072505245696,Of the 7 people I personally know with covid-1...,third-party,0,1,0,0
3,1281225112709148672,@Yorks94 @SooozeKerr @LPryceless @SkyNewsBreak...,third-party,0,1,0,0
4,1239282192729231362,Bought an 18-pack of corona. So far only hea...,primary,0,0,1,0


In [ ]:
testset['tweet'][4]

'Bought an 18-pack of corona.   So far only headache. No fever or shortness of breath.   Will update if any further developments.'

In [ ]:
testset['prediction'] = ''

In [ ]:
testset.head()

,id,tweet,labels,non-reporting,third-party,primary,secondary,prediction
0,1271932435790532611,Went ahead and took a COVID test as a precauti...,non-reporting,1,0,0,0,
1,1268564176424759301,@AmericanAir my flight is being sent to a US A...,third-party,0,1,0,0,
2,1280616072505245696,Of the 7 people I personally know with covid-1...,third-party,0,1,0,0,
3,1281225112709148672,@Yorks94 @SooozeKerr @LPryceless @SkyNewsBreak...,third-party,0,1,0,0,
4,1239282192729231362,Bought an 18-pack of corona. So far only hea...,primary,0,0,1,0,


In [ ]:
# testset.columns = 
# testset = testset[['id',	'tweet','prediction','label',	'ProVax'	,'Neutral',	'AntiVax']]
testset = testset[['id',	'tweet','prediction','labels',	'non-reporting',	'third-party',	'primary',	'secondary']]

In [ ]:
testset.head()

,id,tweet,prediction,labels,non-reporting,third-party,primary,secondary
0,1271932435790532611,Went ahead and took a COVID test as a precauti...,,non-reporting,1,0,0,0
1,1268564176424759301,@AmericanAir my flight is being sent to a US A...,,third-party,0,1,0,0
2,1280616072505245696,Of the 7 people I personally know with covid-1...,,third-party,0,1,0,0
3,1281225112709148672,@Yorks94 @SooozeKerr @LPryceless @SkyNewsBreak...,,third-party,0,1,0,0
4,1239282192729231362,Bought an 18-pack of corona. So far only hea...,,primary,0,0,1,0


In [ ]:
len(testset)

472

In [ ]:
# train_data_loader1 = torch.utils.data.DataLoader(train_df, 
#     batch_size=TRAIN_BATCH_SIZE,
#     shuffle=True,
#     num_workers=0
# )

# val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
#     batch_size=VALID_BATCH_SIZE,
#     shuffle=False,
#     num_workers=0
# )

In [ ]:
# # from simpletransformers.classification import ClassificationModel
# from sklearn.model_selection import KFold
# from sklearn.metrics import accuracy_score
# n=5
# kf = KFold(n_splits=n, random_state=2, shuffle=True)

# results = []

# for train_index, val_index in kf.split(testset):
#     		# splitting Dataframe (dataset not included)
#         train_df2 = testset.iloc[train_index]
#         val_df2 = testset.iloc[val_index]

#         train_dataset2 = CustomDataset(train_df2, tokenizer, MAX_LEN)
#         valid_dataset2 = CustomDataset(val_df2, tokenizer, MAX_LEN)

#         train_data_loader1 = torch.utils.data.DataLoader(train_dataset2, 
#         batch_size=TRAIN_BATCH_SIZE,
#         shuffle=True,
#         num_workers=0)

#         val_data_loader1 = torch.utils.data.DataLoader(valid_dataset2, 
#         batch_size=VALID_BATCH_SIZE,
#         shuffle=False,
#         num_workers=0)

#         # val_targets=[]
#         # val_outputs=[]

#         trained_model = train_model(EPOCHS, train_data_loader1, val_data_loader1, model, optimizer, ckpt_path, best_model_path)

# # # train_df.head()       

# #         for i in range(len(val_df2)):

# #           encodings = tokenizer.encode_plus(
# #           val_df2['tweet'][i],
# #           None,
# #           add_special_tokens=True,
# #           max_length=MAX_LEN,
# #           padding='max_length',
# #           return_token_type_ids=True,
# #           truncation=True,
# #           return_attention_mask=True,
# #           return_tensors='pt'
# #         )

# #         model.eval()
# #         with torch.no_grad():
# #           input_ids = encodings['input_ids'].to(device, dtype=torch.long)
# #           attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
# #           token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
# #           output = model(input_ids, attention_mask, token_type_ids)
# #           final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
# #           print(final_output)
# #           print(val_df2.columns[4:].to_list()[int(np.argmax(final_output, axis=1))])
# #           val_df2['prediction'][i] = val_df2.columns[4:].to_list()[int(np.argmax(final_output, axis=1))]
# #           acc = accuracy_score(val_df2['prediction'], val_df2['label'])
# #           results.append(acc)
# #           print(acc)
# # print(results)



############# Epoch 1: Training Start   #############


KeyError: ignored

In [ ]:
for i in range(472):
  encodings = tokenizer.encode_plus(
    testset['tweet'][i],
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
  )

  model.eval()
  with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(final_output)
    print(testset.columns[4:].to_list()[int(np.argmax(final_output, axis=1))])
    testset['prediction'][i] = testset.columns[4:].to_list()[int(np.argmax(final_output, axis=1))]


  



[[0.984265923500061, 0.011466169729828835, 0.015856288373470306, 0.019718661904335022]]
non-reporting
[[0.03593401238322258, 0.9723746180534363, 0.035479336977005005, 0.040794361382722855]]
third-party
[[0.031884390860795975, 0.9560084342956543, 0.044448185712099075, 0.03789950907230377]]
third-party


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[[0.024157991632819176, 0.9427360892295837, 0.05281415209174156, 0.047805555164813995]]
third-party
[[0.018680419772863388, 0.030055556446313858, 0.9645793437957764, 0.014861281961202621]]
primary
[[0.01874961517751217, 0.014777387492358685, 0.978980541229248, 0.02275172807276249]]
primary
[[0.9868558049201965, 0.01201197225600481, 0.009834080934524536, 0.016341453418135643]]
non-reporting
[[0.9844180345535278, 0.012192370370030403, 0.017753778025507927, 0.0152499470859766]]
non-reporting
[[0.024529021233320236, 0.020086202770471573, 0.9801034331321716, 0.017470676451921463]]
primary
[[0.9800971150398254, 0.008628985844552517, 0.021436071023344994, 0.012660632841289043]]
non-reporting
[[0.9861714243888855, 0.012865728698670864, 0.015612956136465073, 0.014506081119179726]]
non-reporting
[[0.0398130938410759, 0.980138897895813, 0.04026729613542557, 0.03961072117090225]]
third-party
[[0.02747378684580326, 0.016143957152962685, 0.9807252287864685, 0.016672568395733833]]
primary
[[0.0564413

In [ ]:
# testset['temp'] = testset['prediction']
# testset['prediction'] = testset['label']



In [ ]:
testset.tail(20)

,id,tweet,prediction,labels,non-reporting,third-party,primary,secondary
452,1239785329726345216,is the corona virus causing my cough or it jus...,primary,primary,0,0,1,0
453,1249060938675716096,Staying at home saves lives You can help stop ...,non-reporting,non-reporting,1,0,0,0
454,1242239090147364867,Catch corona den cough ina opp face ‼️,non-reporting,non-reporting,1,0,0,0
455,1244187983193894912,Coronavirus Update: Not all Cold and Cough are...,non-reporting,non-reporting,1,0,0,0
456,1262611228796436480,@laxattitude it is linked to inflammation of f...,non-reporting,non-reporting,1,0,0,0
457,1254415309261271041,@arkmech11 @news7tamil @PMOIndia @CMOTamilNadu...,non-reporting,non-reporting,1,0,0,0
458,1297888365027528705,@nagarkoti @timesofindia I was found corona po...,primary,primary,0,0,1,0
459,1272157075045601280,can’t cough or sneeze round Khaza he gone swea...,non-reporting,non-reporting,1,0,0,0
460,1242008915258363904,@MrJerryOC @slidersfanblog @i_sliders @Alexsli...,non-reporting,non-reporting,1,0,0,0
461,1305278470872686594,@lapatronabby Ahh thank you 💛 I’ve been worrie...,primary,primary,0,0,1,0


In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(testset['prediction'], testset['labels'])

0.9957627118644068

In [ ]:
# model.eval()
# with torch.no_grad():
#     input_ids = encodings['input_ids'].to(device, dtype=torch.long)
#     attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
#     token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
#     output = model(input_ids, attention_mask, token_type_ids)
#     final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
#     print(train_df.columns[2:].to_list()[int(np.argmax(final_output, axis=1))])

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/symp_test.csv')

In [ ]:
test_data.tail()

,id,tweet
395,1255420681581404163t,#bbcgms People who don't have COVID-19 cough s...
396,1331420575265394688t,"Fatigue, shortness of breath, muscle aches, he..."
397,1272691373645467653t,I woke up today from my nap and my throat was ...
398,1242065925337018369t,"#coronavirusindia It's just beginning, isolati..."
399,1314084455741632512t,"@WWomenL @TheyWereStolen You beat me to it, co..."


In [ ]:
test_data['prediction'] = ''

In [ ]:
test_data.head()

,id,tweet,prediction
0,1242353332817137664t,If you think you have been exposed to COVID-19...,
1,1244712715827982339t,"@timbovisser @BBCJohnBeattie Yes, myself, wife...",
2,1314693013810089984t,Lots of people I know having positive covid te...,
3,1307860995692527616t,@lizzyp144 @TomAThakkar Okay but doesn’t COVID...,
4,1277375421164613632t,We should have people w covid cough on white b...,


In [ ]:
test_data['non-reporting'] = ''
test_data['third-party'] = ''
test_data['primary'] = ''
test_data['secondary'] = ''

In [ ]:
test_data.head()

,id,tweet,prediction,non-reporting,third-party,primary,secondary
0,1242353332817137664t,If you think you have been exposed to COVID-19...,,,,,
1,1244712715827982339t,"@timbovisser @BBCJohnBeattie Yes, myself, wife...",,,,,
2,1314693013810089984t,Lots of people I know having positive covid te...,,,,,
3,1307860995692527616t,@lizzyp144 @TomAThakkar Okay but doesn’t COVID...,,,,,
4,1277375421164613632t,We should have people w covid cough on white b...,,,,,


In [ ]:
len(test_data)

400

In [ ]:
for i in range(400):
  encodings = tokenizer.encode_plus(
    test_data['tweet'][i],
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
  )

  model.eval()
  with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(final_output)
    print(test_data.columns[3:].to_list()[int(np.argmax(final_output, axis=1))])
    test_data['prediction'][i] = test_data.columns[3:].to_list()[int(np.argmax(final_output, axis=1))]


  



[[0.9859765768051147, 0.012203807942569256, 0.010617335326969624, 0.013685164041817188]]
non-reporting
[[0.02512841671705246, 0.0398765429854393, 0.05141204595565796, 0.795880138874054]]
secondary
[[0.019680924713611603, 0.9510995745658875, 0.024849046021699905, 0.07024671137332916]]
third-party
[[0.9712991714477539, 0.025566088035702705, 0.008963502943515778, 0.013095954433083534]]
non-reporting
[[0.9786157608032227, 0.020357586443424225, 0.010815070942044258, 0.011320844292640686]]
non-reporting
[[0.018955761566758156, 0.028229188174009323, 0.9721251726150513, 0.014360836707055569]]
primary
[[0.9866587519645691, 0.013372537679970264, 0.011931126937270164, 0.013346229679882526]]
non-reporting
[[0.053529467433691025, 0.9731312990188599, 0.052424270659685135, 0.023060409352183342]]
third-party
[[0.9794270992279053, 0.022959712892770767, 0.013782642781734467, 0.016892606392502785]]
non-reporting
[[0.01009423565119505, 0.02301030419766903, 0.19751934707164764, 0.7769568562507629]]
seconda

In [ ]:
test_data.head(42)

,id,tweet,prediction,non-reporting,third-party,primary,secondary
0,1242353332817137664t,If you think you have been exposed to COVID-19...,non-reporting,,,,
1,1244712715827982339t,"@timbovisser @BBCJohnBeattie Yes, myself, wife...",secondary,,,,
2,1314693013810089984t,Lots of people I know having positive covid te...,third-party,,,,
3,1307860995692527616t,@lizzyp144 @TomAThakkar Okay but doesn’t COVID...,non-reporting,,,,
4,1277375421164613632t,We should have people w covid cough on white b...,non-reporting,,,,
5,1238141304187617284t,"So, our rep just announced they are out of ha...",primary,,,,
6,1298691905950232577t,You can integrate technologies you already own...,non-reporting,,,,
7,1238645951356489729t,#BREAKING NEWS ... John Travolta was hospital...,third-party,,,,
8,1242336452056190976t,@WHO Hi in tamil medicine we use this herb (Le...,non-reporting,,,,
9,1328318630380101632t,@rainnwilson Can anybody tell me what symptoms...,secondary,,,,


In [ ]:
test_data.drop(['tweet',	'non-reporting',	'third-party'	,'primary','secondary'], axis = 1, inplace = True)

In [ ]:
test_data.head()

,id,prediction
0,1242353332817137664t,non-reporting
1,1244712715827982339t,secondary
2,1314693013810089984t,third-party
3,1307860995692527616t,non-reporting
4,1277375421164613632t,non-reporting


In [ ]:
test_data.columns = ['id', 'pred']

In [ ]:
test_data.to_csv('akshit_rohit_symp.csv', header = True, index = False)